## LIBRERIAS y CONSTANTES

In [1]:
import numpy as np
import pandas as pd
import pickle
import torch.nn as nn
import torch
from torch.utils.data import DataLoader
import json, ast, sys, csv, random
import plotly.express as px
import math
import datetime
import sys
import csv

#Implement training process
#from model_trees_algebra import NeoRegression

from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from functions.tree_format import IterateBuildTree, InnerJoinsIntraBGPS, \
                                IterateBuildTreeBetweenBGPS, TreeFormat, TreeFormat_all

from functions.aux import MetricTotalAccuraccy
# Load tree transformation
from functions.transform_inference import TransformerInference

from net import NeoNet

class BaoTrainingException(Exception):
    pass
csv.field_size_limit(sys.maxsize)


IS CUDA AVAILABLE: True


131072

### Basic parameters

In [2]:
URL = "/media/data/ccarmona/memoria/dataset/"
csv_name = 'new_dataset_7.2_subqueries'
model_name = 'Model_Adam_0.000015'
model_path = "./" + model_name + '.pt'
x = [True,False]
active_new_data = x[0]
symbol = "ᶲ"
optimizer = "Adam"
#Este parametro sirve para elegir cierta cantidad de data ordenado por rangos de tiempo obtenidos.
## Entre más bajo menos data se seleccionara. Si es muy alto se tendran demasiados valores outliners, 
## pero si es muy bajo podría tenerse una data no representativa y se aumenta el riesgo de overfitting.
## Por otro lado min_data, simplemente da el valor minimo de tiempo de ejecución que tiene una consulta tomada
## en cuenta para hacer el modelo
#percent_of_data_or = 0.93
#min_time_or = 15
#max_time_or = 80
#percent_of_data = 1
#min_time = 5
#max_time = 80


In [3]:
epsilon = 0.75 ## EPSILON
lr_rl = 0.999 ## LEARNING RATE RL
disc_fac = 0.999 ## DISCOUNT FACTOR RL 
epoch_rl = 200 ### TOTAL EPOCH POR DATA RL
n = model_name + str(lr_rl)##SALT Nombre archivos

### Features

In [4]:
# Columns to use.
list_columns = ['total_bgps', 'triples', 'treesize', 'join', 'left_join']

In [5]:
ds_train = pd.read_csv(URL + csv_name + '_ds_train_ft.csv', engine='python', encoding='utf-8')
ds_val = pd.read_csv(URL + csv_name + '_ds_val_ft.csv', engine='python', encoding='utf-8')
ds_test = pd.read_csv(URL + csv_name + '_ds_test_ft.csv', engine='python', encoding='utf-8')
ds_rl = pd.read_csv(URL + csv_name + '_ds_rl_ft.csv', engine='python', encoding='utf-8')

In [6]:
print("ds_train.shape",ds_train.shape)
print("ds_val.shape",ds_val.shape)
print("ds_test.shape",ds_test.shape)
print("ds_rl.shape",ds_rl.shape)

ds_train.shape (6849, 15)
ds_val.shape (1476, 15)
ds_test.shape (1479, 15)
ds_rl.shape (1845, 67)


# FUNCTIONS

In [7]:
def getpredictions_info(x_val_tree, x_val_query, y_val):
    """
    Get statistics by a set of data. Need the previous trained model(availablre  form reg object).
    :param x_val_tree: Plan level features.
    :param x_val_query: Query level features.
    :param y_val: Real execution time
    :return: Dict with predictions and metrics (mae, rmse, mse)
    """
    Xt, Xq, Yv = reg.json_loads(x_val_tree, x_val_query.values, y_val)
    Xt = [reg.fix_tree(x) for x in Xt]
    Xt = reg.tree_transform.transform(Xt)

    pairs_val = list(zip(list(zip(Xt, Xq)), Yv))
    dataset_val = DataLoader(pairs_val, batch_size=batch_size, num_workers=0, shuffle=False, collate_fn=reg.collate_with_card)
    results_val = reg.predict_best(dataset_val)
    y_pred_val, y_real_val = zip(*results_val)
    mseval = mean_squared_error(y_real_val, y_pred_val)
    maeval = mean_absolute_error(y_real_val, y_pred_val)
    rmseval = np.sqrt(mseval)
    return {"pred": y_pred_val, "real" : y_real_val, "mse": mseval, "mae": maeval, "rmse": rmseval, "history": reg.history}

def getpredictions_info_nojc(x_val_tree, x_val_query, y_val):
    """
    Get statistics by a set of data. Need the previous trained model(availablre  form reg object).
    :param x_val_tree: Plan level features.
    :param x_val_query: Query level features.
    :param y_val: Real execution time
    :return: Dict with predictions and metrics (mae, rmse, mse)
    """
    Xt, Xq, Yv = reg.json_loads(x_val_tree, x_val_query.values, y_val)
    Xt = [reg.fix_tree(x) for x in Xt]
    Xt = reg.tree_transform.transform(Xt)

    pairs_val = list(zip(list(zip(Xt, Xq)), Yv))
    dataset_val = DataLoader(pairs_val, batch_size=batch_size, num_workers=0, shuffle=False, collate_fn=reg.collate)
    results_val = reg.predict_best(dataset_val)
    y_pred_val, y_real_val = zip(*results_val)
    mseval = mean_squared_error(y_real_val, y_pred_val)
    maeval = mean_absolute_error(y_real_val, y_pred_val)
    rmseval = np.sqrt(mseval)
    return {"pred": y_pred_val, "real" : y_real_val, "mse": mseval, "mae": maeval, "rmse": rmseval, "history": reg.history}



def getmax(x):
    lista=  list(x.values())
    maximo = 0
    for el in lista:
        if (maximo < float(el)):
            maximo = float(el)
    return maximo

def pred2index_dict(x, pred_to_index, maxcardinality):
    """
    get histogram from cardinality features. the values is normalized using the max cardinality of predicate in dataset.
    :param x: Tree data from x row sample.
    :param pred_to_index: dict with predicates and their index.
    :param maxcardinality: Max cardiniality in the dataset.
    :return: dictionary with feature json_cardinality.
    """
    resp = {}
    x = json.loads(x)
    for el in x.keys():
        if el in pred_to_index:
            resp[pred_to_index[el]] = float(x[el])/maxcardinality
    return resp

def prepare_query_level_data(x_train_query, x_val_query, x_test_query):
    """ Apply StandardScaller to columns except for json_cardinality that need other proccess"""
    maxcardinality =  x_train_query['json_cardinality'].apply(lambda x: json.loads(x)).apply(lambda x: getmax(x)).max()
    #Scale x_query data.
    xqtrain = x_train_query.drop(columns=['json_cardinality'])
    xqval   = x_val_query.drop(columns=['json_cardinality'])
    xqtest   = x_test_query.drop(columns=['json_cardinality'])

    scalerx = StandardScaler()
    x_train_scaled = scalerx.fit_transform(xqtrain)
    x_val_scaled = scalerx.transform(xqval)
    x_test_scaled = scalerx.transform(xqtest)

    x_train_query =pd.concat([pd.DataFrame(x_train_scaled, index=xqtrain.index, columns=xqtrain.columns),x_train_query[['json_cardinality']]], axis=1)
    x_val_query =  pd.concat([pd.DataFrame(x_val_scaled,   index=xqval.index, columns=xqval.columns),x_val_query[['json_cardinality']]], axis=1)
    x_test_query =  pd.concat([pd.DataFrame(x_test_scaled,   index=xqtest.index, columns=xqtest.columns),x_test_query[['json_cardinality']]], axis=1)

    x_train_query['json_cardinality'] = x_train_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(),maxcardinality))
    x_val_query['json_cardinality'] = x_val_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    x_test_query['json_cardinality'] = x_test_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))

    
    
    return x_train_query, x_val_query, x_test_query

def prepare_query_level_only_json_cardinality(x_train_query, x_val_query, x_test_query, x_rl_query):
    """ Apply StandardScaller to columns except for json_cardinality that need other proccess"""
    maxcardinality =  x_train_query['json_cardinality'].apply(lambda x: json.loads(x)).apply(lambda x: getmax(x)).max()
    
    x_train_query = x_train_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(),maxcardinality))
    x_val_query = x_val_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    x_test_query = x_test_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    x_rl_query = x_rl_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    
    
    return x_train_query, x_val_query, x_test_query, x_rl_query


def prepare_query_level_data_full(x_train_query, x_val_query, x_test_query, x_rl_query):
    """ Apply StandardScaller to columns except for json_cardinality that need other proccess"""
    maxcardinality =  x_train_query['json_cardinality'].apply(lambda x: json.loads(x)).apply(lambda x: getmax(x)).max()
    #Scale x_query data.
    xqtrain = x_train_query.drop(columns=['json_cardinality'])
    xqval   = x_val_query.drop(columns=['json_cardinality'])
    xqtest   = x_test_query.drop(columns=['json_cardinality'])
    xqrl = x_rl_query.drop(columns=['json_cardinality'])
    
    scalerx = StandardScaler()
    x_train_scaled = scalerx.fit_transform(xqtrain)
    x_val_scaled = scalerx.transform(xqval)
    x_test_scaled = scalerx.transform(xqtest)
    x_rl_scaled = scalerx.transform(xqrl)
    
    
    x_train_query = pd.concat([pd.DataFrame(x_train_scaled, index=xqtrain.index, columns=xqtrain.columns),x_train_query[['json_cardinality']]], axis=1)
    x_val_query =  pd.concat([pd.DataFrame(x_val_scaled,   index=xqval.index, columns=xqval.columns),x_val_query[['json_cardinality']]], axis=1)
    x_test_query =  pd.concat([pd.DataFrame(x_test_scaled,   index=xqtest.index, columns=xqtest.columns),x_test_query[['json_cardinality']]], axis=1)
    x_rl_query = pd.concat([pd.DataFrame(x_rl_scaled,   index=xqrl.index, columns=xqrl.columns),x_rl_query[['json_cardinality']]], axis=1)
    
    
    x_train_query['json_cardinality'] = x_train_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(),maxcardinality))
    x_val_query['json_cardinality'] = x_val_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    x_test_query['json_cardinality'] = x_test_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    x_rl_query['json_cardinality'] = x_rl_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    
    
    return x_train_query, x_val_query, x_test_query, x_rl_query

def prepare_query_level_data_full_no_jc(x_train_query, x_val_query, x_test_query, x_rl_query):
    """ Apply StandardScaller to columns except for json_cardinality that need other proccess"""
    scalerx = StandardScaler()
    columns_train, index_train = x_train_query.columns, x_train_query.index
    columns_val, index_val = x_val_query.columns, x_val_query.index
    columns_test, index_test = x_test_query.columns, x_test_query.index
    columns_rl, index_rl = x_rl_query.columns, x_rl_query.index
    
    x_train_scaled = scalerx.fit_transform(x_train_query)
    x_val_scaled = scalerx.transform(x_val_query)
    x_test_scaled = scalerx.transform(x_test_query)
    x_rl_scaled = scalerx.transform(x_rl_query)
    
    x_train_query = pd.DataFrame(x_train_scaled,   index=index_train, columns=columns_test)
    x_val_query = pd.DataFrame(x_val_scaled,   index=index_val, columns=columns_val)
    x_test_query = pd.DataFrame(x_test_scaled,   index=index_test, columns=columns_test)
    x_rl_query = pd.DataFrame(x_rl_scaled,   index=index_rl, columns=columns_rl)
    return x_train_query, x_val_query, x_test_query, x_rl_query



### TreeConv Neural Net

In [8]:
folds_execution = {}
print("Size Train: {}, Val {}".format(ds_train.shape[0], ds_val.shape[0]))

Size Train: 6849, Val 1476


In [9]:
# get query level data
x_train_query = ds_train[list_columns]
x_val_query   = ds_val[list_columns]

x_train_query_str = ds_train['query']
x_val_query_str = ds_train['query']

# get plan level datba
x_train_tree = ds_train['trees'].values
x_val_tree = ds_val['trees'].values

y_train = ds_train['time'].values
y_val = ds_val['time'].values

x_test_tree = ds_test['trees'].values
y_test = ds_test['time'].values
x_test_query   = ds_test[list_columns]

x_rl_tree = ds_rl['trees'].values
y_rl = ds_rl['time'].values
x_rl_query   = ds_rl[list_columns]

In [10]:
print("---------SHAPES-----------")
print("----------CLEAN-----------")
print(f'shape ds_train: {ds_train.shape}')
print(f'shape ds_val  : {ds_val.shape}')
print(f'shape ds_test : {ds_test.shape}')
print(f'shape ds_rl : {ds_rl.shape}')
print("")
print("-----TRAIN AND VAL DATA-----")
print("----------x_query_data----------")
print(f'shape x_val_query  : {x_val_query.shape}')
print(f'shape x_train_query: {x_train_query.shape}')
print("----------x_plan_level_data----------")
print(f'shape x_val_tree  : {x_val_tree.shape}')
print(f'shape x_train_tree: {x_train_tree.shape}')
print("----------y_data------------")
print(f'shape y_val  : {y_val.shape}')
print(f'shape y_train: {y_train.shape}')
print("")
print("----------TEST DATA----------")
print(f'shape x_test_tree : {x_test_tree.shape}')
print(f'shape x_test_query: {x_test_query.shape}')
print(f'shape y_test      : {y_test.shape}')
print("-----------------------")
print("")
print("----------RL DATA----------")
print(f'shape x_rl_tree : {x_rl_tree.shape}')
print(f'shape x_rl_query: {x_rl_query.shape}')
print(f'shape y_rl      : {y_rl.shape}')
print("-----------------------")


---------SHAPES-----------
----------CLEAN-----------
shape ds_train: (6849, 15)
shape ds_val  : (1476, 15)
shape ds_test : (1479, 15)
shape ds_rl : (1845, 67)

-----TRAIN AND VAL DATA-----
----------x_query_data----------
shape x_val_query  : (1476, 5)
shape x_train_query: (6849, 5)
----------x_plan_level_data----------
shape x_val_tree  : (1476,)
shape x_train_tree: (6849,)
----------y_data------------
shape y_val  : (1476,)
shape y_train: (6849,)

----------TEST DATA----------
shape x_test_tree : (1479,)
shape x_test_query: (1479, 5)
shape y_test      : (1479,)
-----------------------

----------RL DATA----------
shape x_rl_tree : (1845,)
shape x_rl_query: (1845, 5)
shape y_rl      : (1845,)
-----------------------


In [11]:
maxcardinality = 0
#maxcardinality =  x_train_query['json_cardinality'].apply(lambda x: json.loads(x)).apply(lambda x: getmax(x)).max()
maxcardinality

0

In [12]:
data_inference = {}
bao_base = ""
with (open(bao_base + "preprocess_inference_info_" + model_name + ".pickle", "rb")) as openfile:
    while True:
        try:
            data_inference = pickle.load(openfile)
        except EOFError:
            break
# Load Scaler for query features.
scalerx = None
with (open(bao_base + "scalerx_" + model_name + ".pkl", "rb")) as openfile:
    while True:
        try:
            scalerx = pickle.load(openfile)
        except EOFError:
            break

pipeline_inverse = None
with (open(bao_base + "pipeline_inverse_" + model_name + ".pkl", "rb")) as openfile:
    while True:
        try:
            pipeline_inverse = pickle.load(openfile)
        except EOFError:
            break
with open('NeoNetValues_' + model_name +'.txt', 'r') as file:
    data = file.read().rstrip()
neo_data = ast.literal_eval(data)

IS CUDA AVAILABLE: True


In [13]:
def ActivationFuncReluOrLeaky(string):
    if string == "<class 'torch.nn.modules.activation.LeakyReLU'>":
        return nn.LeakyReLU
    elif string == "<class 'torch.nn.modules.activation.ReLU'>":
        return nn.ReLU
io_dim_model_data = int(neo_data['io_dim'])
query_input_size = int(neo_data['query_input_size'])
query_hidden_inputs = ast.literal_eval(neo_data['query_hidden_inputs'])
query_output = int(neo_data['query_output'])
tree_units = ast.literal_eval(neo_data['tree_units'])
tree_units_dense = ast.literal_eval(neo_data['tree_units_dense'])
tree_activation_tree = ActivationFuncReluOrLeaky(neo_data['activation_tree'])
tree_activation_dense = ActivationFuncReluOrLeaky(neo_data['activation_dense'])

def get_prediction_dataset_from_load(ds, list_columns):
    x_tree = ds["trees"].values
    x_query = ds[list_columns]
    dataset_val = transformer.prepare(x_tree, x_query)
    predictions, predictions_extend = transformer.getpredictions_info(dataset_val)
    predictions_extend_full = [i[0] for i in predictions_extend]
    targets = list(ds["time"].values)
    mseval = mean_squared_error(targets, predictions_extend_full)
    maeval = mean_absolute_error(targets, predictions_extend_full)
    rmseval = np.sqrt(mseval)
    return {"pred": predictions_extend_full, "real" : targets, "mse": mseval, "mae": maeval, "rmse": rmseval}

def get_prediction_from_load(x_tree, x_query, targets):
    dataset_val = transformer.prepare(x_tree, x_query)
    predictions, predictions_extend = transformer.getpredictions_info(dataset_val)
    predictions_extend_full = [i[0] for i in predictions_extend]
    mseval = mean_squared_error(targets, predictions_extend_full)
    maeval = mean_absolute_error(targets, predictions_extend_full)
    rmseval = np.sqrt(mseval)
    return {"pred": predictions_extend_full, "real" : targets, "mse": mseval, "mae": maeval, "rmse": rmseval}


In [14]:
reg = NeoNet(
    io_dim_model_data,
    query_input_size,
    query_hidden_inputs,
    query_output=query_output,
    tree_units=tree_units,
    tree_units_dense=tree_units_dense,
    activation_tree=tree_activation_tree,
    activation_dense=tree_activation_dense,
)
device = torch.device("cuda")
# Loading model parameters.
reg_path = model_path
#reg.load_state_dict(torch.load(reg_path, map_location=torch.device('cpu')))
reg.load_state_dict(torch.load(reg_path))
reg.to(device)
transformer = TransformerInference(
        model=reg,
        tree_transform=data_inference["tree_transform"],
        pipeline_inverse=pipeline_inverse,
        scalerx=scalerx,
        maxcardinality=data_inference["maxcardinality"]
    )
print("Modelos Listos")

Activation function in treeConv layers: <class 'torch.nn.modules.activation.LeakyReLU'>
Activation function in tree dense output layers: <class 'torch.nn.modules.activation.ReLU'>
[BinaryTreeConvWithQData(
  (weights): Conv1d(1282, 512, kernel_size=(3,), stride=(3,))
), TreeLayerNorm(), TreeActivation(
  (activation): LeakyReLU(negative_slope=0.01)
), BinaryTreeConv(
  (weights): Conv1d(512, 256, kernel_size=(3,), stride=(3,))
), TreeLayerNorm(), TreeActivation(
  (activation): LeakyReLU(negative_slope=0.01)
), BinaryTreeConv(
  (weights): Conv1d(256, 128, kernel_size=(3,), stride=(3,))
), TreeLayerNorm(), TreeActivation(
  (activation): LeakyReLU(negative_slope=0.01)
), DynamicPooling(), Linear(in_features=128, out_features=64, bias=True), ReLU(), Linear(in_features=64, out_features=32, bias=True), ReLU(), Linear(in_features=32, out_features=1, bias=True)]
Modelos Listos


In [15]:
 ds_test['time']

0         1.112900
1         1.983621
2         1.503600
3         1.404750
4         1.669234
           ...    
1474    102.446694
1475    148.000000
1476    107.995152
1477    113.000000
1478    102.963054
Name: time, Length: 1479, dtype: float64

In [16]:
results = get_prediction_from_load(ds_test['trees'].values, ds_test[list_columns], ds_test['time'])

In [17]:
results

{'pred': [1.8008136749267578,
  2.7519922256469727,
  17.93634605407715,
  82.53691864013672,
  20.08209228515625,
  15.601694107055664,
  9.666911125183105,
  1.4511525630950928,
  16.62332534790039,
  16.957927703857422,
  27.374305725097656,
  1.3945047855377197,
  15.441661834716797,
  15.28744125366211,
  16.5111026763916,
  1.5850610733032227,
  14.590093612670898,
  1.990408182144165,
  4.276917457580566,
  21.684619903564453,
  16.213970184326172,
  15.517280578613281,
  41.746604919433594,
  4.316860198974609,
  16.394941329956055,
  17.04638671875,
  72.1273193359375,
  6.603542804718018,
  35.03512191772461,
  74.40719604492188,
  4.345771789550781,
  47.48903274536133,
  4.350900173187256,
  16.315570831298828,
  2.093446731567383,
  3.4125804901123047,
  8.490581512451172,
  1.699221134185791,
  14.699965476989746,
  9.989614486694336,
  1.4968600273132324,
  79.6812515258789,
  9.945648193359375,
  16.690385818481445,
  16.783056259155273,
  15.430166244506836,
  4.091226

## Reinforcement Learning

In [18]:
def GetTriplesSubtree(subtree_as_str):
    code_tpf = ['VAR_VAR_VAR', 'VAR_VAR_URI', 'VAR_URI_VAR', 'VAR_URI_URI', 'VAR_URI_LITERAL', 'VAR_VAR_LITERAL',
                'URI_URI_LITERAL', 'URI_URI_VAR', 'URI_URI_URI', 'URI_VAR_VAR', 'URI_VAR_URI', 'URI_VAR_LITERAL',
                'LITERAL_URI_VAR', 'LITERAL_URI_URI', 'LITERAL_URI_LITERAL']
    total_triples = 0
    for i in code_tpf:
        total_triples += subtree_as_str.count(i)
    #if subtree_as_str in code_tpf:
    #    total_triples += 1
    return total_triples

def GetTreeSize(subtrees, treesize):
    if len(subtrees) == 1:
        return treesize
    else:
        treesize += 1
        left_treesize = GetTreeSize(subtrees[1], treesize)
        right_treesize = GetTreeSize(subtrees[2], treesize)
        treesize = max(left_treesize,right_treesize)
    return treesize

def GetAllJoins(subtree_as_str):
    join = subtree_as_str.count('JOIN')
    left_join = subtree_as_str.count('LEFT_JOIN')
    return join-left_join, left_join

def GetIter(subtree_as_str):
    if "iter" in subtree_as_str:
        return 1
    else:
        return 0
    
def GetTotalBgp(state):
    bgp_list = []
    for s in state:
        bgp_list.append(s[1])
    bgp_list = set(bgp_list)
    return len(bgp_list)

def GetDataframe(subtree, state, columns):
    subtree_str = str(subtree)
    subtree_list = ast.literal_eval(subtree[0].tolist())
    total_bgps = GetTotalBgp(state)
    treesize = GetTreeSize(subtree_list, 1)
    triples = GetTriplesSubtree(subtree_str)
    join, left_join = GetAllJoins(subtree_str)
    iters = GetIter(subtree_str)
    
    values = [total_bgps, triples, treesize, join, left_join]
    x_rl_query = pd.DataFrame([values], columns = columns)
    return x_rl_query
    #print(x_rl_query)
    #scalerx = StandardScaler()
    #values_scaled = scalerx.fit_transform(x_rl_query)
    #x_rl_query = pd.DataFrame(values_scaled, columns = columns)
    #print(x_rl_query)
    
def RL_Actions(bgp):
    actions = []
    idx = 0
    for k,v in bgp.items():
        bgp_list = v['bgp_list']
        for b in v['bgp_list']:
            actions.append((idx,k,b['P'],b['triple_type']))
            idx += 1
    return actions


def RL_Initial_Step(actions):
    initial_state = []
    random_index = random.randint(0, len(actions)-1)
    random_action = actions[random_index]
    initial_state.append(random_action)
    return initial_state

def RL_available_actions(actions, current_state):
    if not current_state:
        return actions
    available_actions = sorted(list(set(actions) - set(current_state)))
    same_bgp_actions = []
    other_bgp_actions = []
    for i in available_actions:
        if current_state[-1][1] == i[1]:
            same_bgp_actions.append(i)
        else:
            other_bgp_actions.append(i)
    if same_bgp_actions:
        available_actions = same_bgp_actions
    else:
        available_actions = other_bgp_actions
    return available_actions


def RL_Argmax(array):
    argmax_list = np.argwhere(array == np.amax(array))
    argmax_list_flatten = [item for sublist in argmax_list for item in sublist]
    return int(random.choice(argmax_list_flatten))

def RL_Argmax_available(q_value,available_actions):
    available_idx = [i[0] for i in available_actions]
    #print("available_actions",available_actions)
    #print("available_idx",available_idx)
    q_value_available = [q_value[i] for i in available_idx]
    #print("q_values",q_value, type(q_value))
    #print("q_values_available",q_value_available, type(q_value_available))
    
    argmax_list = np.argwhere(q_value_available == np.amax(q_value_available))
    argmax_list_flatten = [item for sublist in argmax_list for item in sublist]
    #print("argmax_list",argmax_list, type(argmax_list))
    #print("argmax_list_flatten",argmax_list_flatten, type(argmax_list_flatten))
    value = int(random.choice(argmax_list_flatten))
    #print("value",value)
    
    #print("..........................")
    return value


def RL_Next_step(actions,
                current_state,
                reward,
                q_value,
                x_rl_tree_ind,
                x_rl_query_ind,
                y_rl_ind,
                bgp_ind,
                epsilon
                ):
    random_num = np.random.random()
    available_actions = RL_available_actions(actions, current_state)
    actual_state = len(current_state) - 1
    
    if random_num < epsilon:
        chosen_action_available_idx = random.randint(0, len(available_actions)-1)
        chosen_action = available_actions[chosen_action_available_idx]
        chosen_action_idx = chosen_action[0]
        #print(chosen_action_idx)
        current_state.append(chosen_action)
        reward, terminal = RL_Reward(actions,
                                     available_actions,
                                     current_state,
                                     chosen_action,
                                     reward,
                                     x_rl_tree_ind,
                                     x_rl_query_ind,
                                     y_rl_ind,
                                     bgp_ind)
    else:
        #print(current_state)
        chosen_action_available_idx = RL_Argmax_available(q_value[actual_state],available_actions)
        chosen_action = available_actions[chosen_action_available_idx]
        chosen_action_idx = chosen_action[0]
        current_state.append(chosen_action)
        reward, terminal = RL_Reward(actions,
                                     available_actions,
                                     current_state,
                                     chosen_action,
                                     reward,
                                     x_rl_tree_ind,
                                     x_rl_query_ind,
                                     y_rl_ind,
                                     bgp_ind)
    return current_state, reward, terminal, chosen_action, chosen_action_idx

def RL_Reward(actions,
              available_actions,
              current_state,
              chosen_action,
              reward,
              x_rl_tree_ind,
              x_rl_query_ind,
              y_rl_ind,
              bgp_ind):
    terminal = False
    #print(available_actions)
    #if chosen_action not in available_actions:
    #    reward -= 100000
    if len(current_state) == len(actions):
        terminal = True
    new_dicto = RL_Rebuild_Dictionary(bgp_ind, current_state)
    #tree_format = TreeFormat_all(new_dicto,symbol)
    tree_format = TreeFormat(new_dicto,symbol)
    #tree_format = ast.literal_eval(tree_format)
    new_tree = np.array([str(tree_format).replace('"', ';').replace("'", '"')])
    x_rl_query = GetDataframe(new_tree, current_state, x_rl_query_ind.columns)
    pred = get_prediction_from_load(new_tree, x_rl_query, y_rl_ind)
    reward -= pred['pred'][0]
    return reward, terminal

def RL_Rebuild_Dictionary(bgp, final_state):
    new_dicto = {}
    bgp_names = list(set([i[1] for i in final_state]))
    ### Keys
    for i in bgp_names:
        new_dicto[i] = {"bgp_list" : [], "opt" : bgp[i]['opt']}
    for i in final_state:
        new_dicto[i[1]]["bgp_list"].append({'P' : i[2], 'triple_type' : i[3]})

    return new_dicto


def RL_First_Policy(actions):
    actions_length = len(actions)
    return np.zeros((actions_length,actions_length))

def RL_bgp_format(ds_rl):
    bgps_rl = ds_rl['bgps'].values
    bgps_rl = [ast.literal_eval(bgp) for bgp in bgps_rl]
    return bgps_rl

def RL_get_data(x_rl_tree, x_rl_query, y_rl, bgps_rl,idx):
    x_rl_tree_c = x_rl_tree.copy()
    x_rl_query_c = x_rl_query.copy()
    y_rl_c = y_rl.copy()
    bgps_rl_c = bgps_rl.copy()
    x_rl_query_c = x_rl_query_c.reset_index(drop=True)
    columns = x_rl_query_c.columns
    values = x_rl_query_c.values[idx]    
    x_rl_tree_test = np.array([x_rl_tree_c[idx]])
    x_rl_query_test = pd.DataFrame([values],columns=columns)
    y_rl_test = np.array([y_rl_c[idx]])
    bgps_test = bgps_rl_c[idx]
    return x_rl_tree_test, x_rl_query_test, y_rl_test, bgps_test


def RL_get_final_state_bgp_tree(q_value,actions,bgp,symbol):
    q_value = q_value.tolist()
    best_state = []
    arg_max = RL_Argmax(q_value[0])
    best_state.append(actions[arg_max])
        
    for q_val in q_value[1:]:
        available_actions = RL_available_actions(actions, best_state)
        chosen_action_available_idx = RL_Argmax_available(q_val,available_actions)
        chosen_action = available_actions[chosen_action_available_idx]
        chosen_action_idx = chosen_action[0]
        best_state.append(chosen_action) 
    new_dicto = RL_Rebuild_Dictionary(bgp, best_state)
    #new_tree = np.array([str(TreeFormat_all(new_dicto,symbol)).replace('"', ';').replace("'", '"')])
    new_tree = np.array([str(TreeFormat(new_dicto,symbol)).replace('"', ';').replace("'", '"')])
    
    return best_state, new_dicto, new_tree


def RL_results_functions(idx,pred_old, pred_new,prl_tol1,prl_tol2):
    difference = pred_new - pred_old
    abs_diff = np.abs(difference)
    best_of_the_pred = []
    similar_of_the_pred = []
    worst_of_the_pred = []
    if abs_diff < prl_tol1:
        best_of_the_pred.append(idx)
    elif abs_diff < prl_tol2:
        similar_of_the_pred.append(idx)
    else:
        worst_of_the_pred.append(idx)
    return best_of_the_pred, similar_of_the_pred, worst_of_the_pred
    
def RLNeo_Next_step(actions,
                current_state,
                reward,
                q_value,
                x_rl_tree_ind,
                x_rl_query_ind,
                y_rl_ind,
                bgp_ind,
                epsilon
                ):
    random_num = np.random.random()
    available_actions = RL_available_actions(actions, current_state)
    actual_state = len(current_state) - 1
    
    if random_num < epsilon:
        chosen_action_available_idx = random.randint(0, len(available_actions)-1)
        chosen_action = available_actions[chosen_action_available_idx]
        chosen_action_idx = chosen_action[0]
        #print(chosen_action_idx)
        current_state.append(chosen_action)
        reward, terminal = RLNeo_Reward(actions,
                                     available_actions,
                                     current_state,
                                     chosen_action,
                                     reward,
                                     x_rl_tree_ind,
                                     x_rl_query_ind,
                                     y_rl_ind,
                                     bgp_ind)
    else:
        #print(current_state)
        chosen_action_available_idx = RL_Argmax_available(q_value[actual_state],available_actions)
        chosen_action = available_actions[chosen_action_available_idx]
        chosen_action_idx = chosen_action[0]
        current_state.append(chosen_action)
        reward, terminal = RLNeo_Reward(actions,
                                     available_actions,
                                     current_state,
                                     chosen_action,
                                     reward,
                                     x_rl_tree_ind,
                                     x_rl_query_ind,
                                     y_rl_ind,
                                     bgp_ind)
    return current_state, reward, terminal, chosen_action, chosen_action_idx

def RLNeo_Reward(actions,
              available_actions,
              current_state,
              chosen_action,
              reward,
              x_rl_tree_ind,
              x_rl_query_ind,
              y_rl_ind,
              bgp_ind):
    terminal = False
    reward = 0
    #print(available_actions)
    #if chosen_action not in available_actions:
    #    reward -= 100000
    if len(current_state) == len(actions):
        terminal = True
        new_dicto = RL_Rebuild_Dictionary(bgp_ind, current_state)
        tree_format = TreeFormat(new_dicto,symbol)
        #tree_format = ast.literal_eval(tree_format)
        new_tree = np.array([str(tree_format).replace('"', ';').replace("'", '"')])
        x_rl_query = GetDataframe(new_tree, current_state, x_rl_query_ind.columns)
        pred = get_prediction_from_load(new_tree, x_rl_query, y_rl_ind)
        reward -= pred['pred'][0]
    return reward, terminal
    
    
def RLNeo_q_values(q_value_neo,current_state, reward):
    for q_val, act in zip(q_value_neo,current_state):
        if q_val[act[0]] == 0:
            q_val[act[0]] = reward
        else:
            q_val[act[0]] = max(reward,q_val[act[0]])
    return q_value_neo

def RL_max_available_actions(q_value, old_states, actions):
    available_actions = RL_available_actions(actions, old_states)
    index_av = [i[0] for i in available_actions]
    q_value_acotado = q_value[index_av[0]:index_av[-1]+1]
    return np.max(q_value_acotado)

In [19]:
print("----------RL DATA----------")
print(f'shape ds_rl : {ds_rl.shape}')
print(f'shape x_rl_tree : {x_rl_tree.shape}')
print(f'shape x_rl_query: {x_rl_query.shape}')
print(f'shape y_rl      : {y_rl.shape}')
print("-----------------------")
bgps_rl = RL_bgp_format(ds_rl)

----------RL DATA----------
shape ds_rl : (1845, 67)
shape x_rl_tree : (1845,)
shape x_rl_query: (1845, 5)
shape y_rl      : (1845,)
-----------------------


## MULTI-VAL

##### Seleccionar data

In [20]:
min_num_actions = -1
buenos_idx = []
aaa = []
for i in range(0,x_rl_tree.shape[0]):
    x_rl_tree_ind, x_rl_query_ind, y_rl_ind, bgp_ind = RL_get_data(x_rl_tree, x_rl_query, y_rl, bgps_rl,i)
    actions = RL_Actions(bgp_ind)
    if len(actions) >= min_num_actions:
        aaa.append(y_rl_ind)
        buenos_idx.append(i)
total_buenos_idx = len(buenos_idx)
print("total_idx",total_buenos_idx)
rl_columns = [
            'index',
            'iteration',
            'pred_old',
            'pred_new',
            'real_old',
            'real_new',
            'mse_old',
            'mse_new',
            'mae_old',
            'mae_new',
            'rmse_old',
            'rmse_new',
            'old_tree',
            'new_tree',
            #'history_old',
            #'history_new'
             ]
 
no_history = [
            'index',
            'iteration',
            'pred_old',
            'pred_new',
            'real_old',
            'real_new',
            'mse_old',
            'mse_new',
            'mae_old',
            'mae_new',
            'rmse_old',
            'rmse_new',
            'old_tree',
            'new_tree'
             ]

total_idx 1845


##### Parameters

In [21]:
#epsilon = 0.7 ## EPSILON
#lr_rl = 0.9 ## LEARNING RATE
#disc_fac = 0.999 ## DISCOUNT FACTOR
#epoch_rl = 200 ### TOTAL EPOCH POR DATA
#n = model_name ##SALT Nombre archivos

### Q - Learning - multival

##### Parameters

In [22]:
epsilon_ql = epsilon ## EPSILON
lr_ql = lr_rl ## LEARNING RATE
disc_fac_ql = disc_fac ## DISCOUNT FACTOR
epoch_ql = epoch_rl ### TOTAL EPOCH POR DATA

##### Loop

In [ ]:
values_ql = []
values_final_ql = []
c_ql = 1
print(f'Start: {str(datetime.datetime.now())}')
for idx in buenos_idx:
    x_rl_tree_ind, x_rl_query_ind, y_rl_ind, bgp_ind = RL_get_data(x_rl_tree, x_rl_query, y_rl, bgps_rl,idx)
    actions = RL_Actions(bgp_ind)
    num_actions = len(actions)
    q_value_ql = RL_First_Policy(actions) ##INITIAL POLICITY
    if c_ql % 10 == 0:
        print(f'{str(datetime.datetime.now())} : {c_ql}/{total_buenos_idx}, index {idx} evaluating')
    ####################################Q LEARNING###############################################
    for i in range(0,epoch_ql):
        current_state = []
        reward = 0
        terminal = False
        current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,   
                                                                                        current_state,
                                                                                        reward,
                                                                                        q_value_ql,
                                                                                        x_rl_tree_ind,
                                                                                        x_rl_query_ind,
                                                                                        y_rl_ind,
                                                                                        bgp_ind,
                                                                                        epsilon_ql)
        old_state = len(current_state)-1
        prev_action = chosen_action_idx
        while not terminal:
            old_states_val = current_state.copy()
            current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,
                                                                                            current_state,
                                                                                            reward,
                                                                                            q_value_ql,
                                                                                            x_rl_tree_ind,
                                                                                            x_rl_query_ind,
                                                                                            y_rl_ind,
                                                                                            bgp_ind,
                                                                                            epsilon_ql)
            
            new_state = len(current_state)-1
            old_q_value = q_value_ql[old_state,prev_action]
            q_max = RL_max_available_actions(q_value_ql[new_state,:], old_states_val, actions)
            temporal_difference = reward + disc_fac_ql * q_max - old_q_value
            #temporal_difference = reward + disc_fac_ql * np.max(q_value_ql[new_state,:]) - old_q_value
            new_q_value = old_q_value + lr_ql * temporal_difference
            q_value_ql[old_state,prev_action] = new_q_value
            old_state = new_state
            prev_action = chosen_action_idx
        old_q_value = q_value_ql[old_state,prev_action]
        new_q_value = old_q_value + lr_ql * (reward - old_q_value)
        q_value_ql[old_state,prev_action] = new_q_value
        ################################################################################################
        
        best_state, new_dicto, new_tree = RL_get_final_state_bgp_tree(q_value_ql,actions,bgp_ind,symbol)
        rl_stats_old = get_prediction_from_load(x_rl_tree_ind, x_rl_query_ind, y_rl_ind)
        rl_stats_new = get_prediction_from_load(new_tree, x_rl_query_ind, y_rl_ind)
        
        row = [
                    idx,
                    i,
                    rl_stats_old['pred'][0],
                    rl_stats_new['pred'][0],
                    float(rl_stats_old['real'][0]),
                    float(rl_stats_new['real'][0]), 
                    rl_stats_old['mse'],
                    rl_stats_new['mse'],
                    rl_stats_old['mae'],
                    rl_stats_new['mae'],
                    rl_stats_old['rmse'],
                    rl_stats_new['rmse'],
                    str(x_rl_tree_ind),
                    str(new_tree),
           ]

        values_ql.append(row)
    values_final_ql.append(row)
    c_ql += 1
print("len(values_ql[0])", len(values_ql[0]))
print("len(values_ql)", len(values_ql))

Start: 2021-12-31 03:32:45.261131
2021-12-31 03:33:21.101368 : 10/1845, index 9 evaluating


In [ ]:
ql_df = pd.DataFrame(values_ql,columns=rl_columns)
ql_df_no_history = ql_df[no_history]
ql_df_no_history.to_csv(URL + 'rl_csvs/ql_df_no_history'+str(n)+'.csv',index=False)

In [ ]:
ql_df_final = pd.DataFrame(values_final_ql,columns=rl_columns)
ql_df_final.to_csv(URL + 'rl_csvs/ql_df_final'+str(n)+'.csv',index=False)

In [ ]:
ql_df_no_history

In [ ]:
ql_df_final

## RL-Neo Multival

###### Parameters

In [ ]:
epsilon_neo = epsilon ## EPSILON
epoch_neo = epoch_rl ### TOTAL EPOCH POR DATA

###### Loop

In [ ]:
values_neo = []
values_final_neo = []
c_neo = 1
print(f'Start: {str(datetime.datetime.now())}')
for idx in buenos_idx:
    x_rl_tree_ind, x_rl_query_ind, y_rl_ind, bgp_ind = RL_get_data(x_rl_tree, x_rl_query, y_rl, bgps_rl,idx)
    actions = RL_Actions(bgp_ind)
    num_actions = len(actions)
    q_value_neo = RL_First_Policy(actions) ##INITIAL POLICITY
    if c_neo % 10 == 0:
        print(f'{str(datetime.datetime.now())} : {c_neo}/{total_buenos_idx}, index {idx} evaluating')
    ############# ALGORITMO NEO #####################3
    for i in range(0,epoch_neo):
        current_state = []
        reward = 0
        terminal = False
        current_state, reward, terminal, chosen_action, chosen_action_idx = RLNeo_Next_step(actions,
                                                                                        current_state,
                                                                                        reward,
                                                                                        q_value_neo,
                                                                                        x_rl_tree_ind,
                                                                                        x_rl_query_ind,
                                                                                        y_rl_ind,
                                                                                        bgp_ind,
                                                                                        epsilon_neo)    
        while not terminal:
            current_state, reward, terminal, chosen_action, chosen_action_idx = RLNeo_Next_step(actions,
                                                                                            current_state,
                                                                                            reward,
                                                                                            q_value_neo,
                                                                                            x_rl_tree_ind,
                                                                                            x_rl_query_ind,
                                                                                            y_rl_ind,
                                                                                            bgp_ind,
                                                                                            epsilon_neo)
        q_value_neo = RLNeo_q_values(q_value_neo,current_state, reward)
        ##############################################################
        best_state, new_dicto, new_tree = RL_get_final_state_bgp_tree(q_value_neo,actions,bgp_ind,symbol)
        rl_stats_old = get_prediction_from_load(x_rl_tree_ind, x_rl_query_ind, y_rl_ind)
        rl_stats_new = get_prediction_from_load(new_tree, x_rl_query_ind, y_rl_ind)
        
        row = [
                    idx,
                    i,
                    rl_stats_old['pred'][0],
                    rl_stats_new['pred'][0],
                    float(rl_stats_old['real'][0]),
                    float(rl_stats_new['real'][0]), 
                    rl_stats_old['mse'],
                    rl_stats_new['mse'],
                    rl_stats_old['mae'],
                    rl_stats_new['mae'],
                    rl_stats_old['rmse'],
                    rl_stats_new['rmse'],
                    str(x_rl_tree_ind),
                    str(new_tree),
           ]
        values_neo.append(row)
    values_final_neo.append(row)
    c_neo += 1
#print("values_neo ", values_neo)
print("len(values_neo[0])",len(values_neo[0]) )
print("len(values_neo)", len(values_neo))

In [ ]:
neo_df = pd.DataFrame(values_neo,columns=rl_columns)
neo_df_no_history = neo_df[no_history]
neo_df_no_history.to_csv(URL + 'rl_csvs/neo_df_no_history'+str(n)+'.csv',index=False)

In [ ]:
neo_df_final = pd.DataFrame(values_final_neo,columns=rl_columns)
neo_df_final.to_csv(URL + 'rl_csvs/neo_df_final'+str(n)+'.csv',index=False)

In [ ]:
neo_df_no_history

In [ ]:
neo_df_final

## Sarsa Multival

###### Parameters

In [ ]:
epsilon_sar = epsilon ## EPSILON
lr_sar = lr_rl ## LEARNING RATE
disc_fac_sar = disc_fac ## DISCOUNT FACTOR
epoch_sar = epoch_rl ### TOTAL EPOCH POR DATA

###### Loop

In [ ]:
values_sar = []
values_final_sar = []
c_sar = 1
print(f'Start: {str(datetime.datetime.now())}')
for idx in buenos_idx:
    x_rl_tree_ind, x_rl_query_ind, y_rl_ind, bgp_ind = RL_get_data(x_rl_tree, x_rl_query, y_rl, bgps_rl,idx)
    actions = RL_Actions(bgp_ind)
    num_actions = len(actions)
    q_value_sar = RL_First_Policy(actions) ##INITIAL POLICITY
    if c_sar % 10 == 0:
        print(f'{str(datetime.datetime.now())} : {c_sar}/{total_buenos_idx}, index {idx} evaluating')
    ############################ALGORITMO SARSA##########################################
    for i in range(0,epoch_sar):
        current_state = []
        reward = 0
        terminal = False
        current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,
                                                                                        current_state,
                                                                                        reward,
                                                                                        q_value_sar,
                                                                                        x_rl_tree_ind,
                                                                                        x_rl_query_ind,
                                                                                        y_rl_ind,
                                                                                        bgp_ind,
                                                                                        epsilon_sar)
        old_state = len(current_state)-1
        prev_action = chosen_action_idx
        while not terminal:
            current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,
                                                                                            current_state,
                                                                                            reward,
                                                                                            q_value_sar,
                                                                                            x_rl_tree_ind,
                                                                                            x_rl_query_ind,
                                                                                            y_rl_ind,
                                                                                            bgp_ind,
                                                                                            epsilon_sar)

            old_q_value = q_value_sar[old_state,prev_action]
            temporal_difference = reward + (disc_fac_sar * q_value_sar[new_state,chosen_action_idx]) - old_q_value
            new_q_value = old_q_value + lr_sar * temporal_difference
            q_value_sar[old_state,prev_action] = new_q_value
            old_state = new_state
            prev_action = chosen_action_idx
        old_q_value = q_value_sar[old_state,prev_action]
        new_q_value = old_q_value + lr_sar * (reward - old_q_value)
        q_value_sar[old_state,prev_action] = new_q_value
        ###############################################################################################33
        best_state, new_dicto, new_tree = RL_get_final_state_bgp_tree(q_value_sar,actions,bgp_ind,symbol)
        rl_stats_old = get_prediction_from_load(x_rl_tree_ind, x_rl_query_ind, y_rl_ind)
        rl_stats_new = get_prediction_from_load(new_tree, x_rl_query_ind, y_rl_ind)
        
        row = [
                    idx,
                    i,
                    rl_stats_old['pred'][0],
                    rl_stats_new['pred'][0],
                    float(rl_stats_old['real'][0]),
                    float(rl_stats_new['real'][0]), 
                    rl_stats_old['mse'],
                    rl_stats_new['mse'],
                    rl_stats_old['mae'],
                    rl_stats_new['mae'],
                    rl_stats_old['rmse'],
                    rl_stats_new['rmse'],
                    str(x_rl_tree_ind),
                    str(new_tree),
           ]

        values_sar.append(row)
    values_final_sar.append(row)
    c_sar += 1
#print("values_sar ", values_sar)
print("len(values_sar[0])",len(values_sar[0]) )
print("len(values_sar)", len(values_sar))

In [ ]:
sarsa_df = pd.DataFrame(values_sar,columns=rl_columns)
sarsa_df_no_history = sarsa_df[no_history]
sarsa_df_no_history.to_csv(URL + 'rl_csvs/sarsa_df_no_history'+str(n)+'.csv',index=False)

In [ ]:
sarsa_df_final = pd.DataFrame(values_final_sar,columns=rl_columns)
sarsa_df_final.to_csv(URL + 'rl_csvs/sarsa_df_final'+str(n)+'.csv',index=False)

In [ ]:
sarsa_df_no_history

In [ ]:
sarsa_df_final

## Expected-Sarsa Multival

###### Parameters

In [ ]:
epsilon_exsar = epsilon ## EPSILON
epsilon_exsar_alg = 0.9
lr_exsar = lr_rl ## LEARNING RATE
disc_fac_exsar = disc_fac ## DISCOUNT FACTOR
epoch_exsar = epoch_rl ### TOTAL EPOCH POR DATA

###### Loop

In [ ]:
values_exsar = []
values_final_exsar = []
c_exsar = 1
print(f'Start: {str(datetime.datetime.now())}')
for idx in buenos_idx:
    x_rl_tree_ind, x_rl_query_ind, y_rl_ind, bgp_ind = RL_get_data(x_rl_tree, x_rl_query, y_rl, bgps_rl,idx)
    actions = RL_Actions(bgp_ind)
    num_actions = len(actions)
    q_value_exsar = RL_First_Policy(actions) ##INITIAL POLICITY
    if c_exsar % 10 == 0:
        print(f'{str(datetime.datetime.now())} : {c_exsar}/{total_buenos_idx}, index {idx} evaluating')
        
    ##### ALGORITMO EXPECTED SARSA
    for i in range(0,epoch_exsar):
        current_state = []
        reward = 0
        terminal = False
        current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,
                                                                                        current_state,
                                                                                        reward,
                                                                                        q_value_exsar,
                                                                                        x_rl_tree_ind,
                                                                                        x_rl_query_ind,
                                                                                        y_rl_ind,
                                                                                        bgp_ind,
                                                                                        epsilon_exsar)    
        old_state = len(current_state)-1
        prev_action = chosen_action_idx
        while not terminal:
            old_states_val = current_state.copy()
            current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,
                                                                                            current_state,
                                                                                            reward,
                                                                                            q_value_exsar,
                                                                                            x_rl_tree_ind,
                                                                                            x_rl_query_ind,
                                                                                            y_rl_ind,
                                                                                            bgp_ind,
                                                                                            epsilon_exsar)
            new_state = len(current_state)-1
            array_state = np.array(current_state)




            expected_q = 0
            old_q_value = q_value_exsar[old_state,prev_action]
            q_max = RL_max_available_actions(q_value_exsar[new_state,:], old_states_val, actions)
            #q_max = np.max(q_value_exsar[new_state,:])
            pi = np.ones(num_actions)*epsilon_exsar_alg/num_actions \
                + (q_value_exsar[new_state,:] == q_max)*(1-epsilon_exsar_alg)/np.sum(q_value_exsar[new_state,:] == q_max)
    
            expected_q = np.sum(q_value_exsar[new_state,:]*pi)
            temporal_difference = reward + disc_fac_exsar * expected_q - old_q_value
            new_q_value = old_q_value +  lr_exsar*temporal_difference
            q_value_exsar[old_state,prev_action] = new_q_value
            old_state = new_state
            prev_action = chosen_action_idx
        #########################################################
        old_q_value = q_value_exsar[old_state,prev_action]
        new_q_value = old_q_value + lr_exsar * (reward - old_q_value)
        q_value_exsar[old_state,prev_action] = new_q_value

        best_state, new_dicto, new_tree = RL_get_final_state_bgp_tree(q_value_exsar,actions,bgp_ind,symbol)
        rl_stats_old = get_prediction_from_load(x_rl_tree_ind, x_rl_query_ind, y_rl_ind)
        rl_stats_new = get_prediction_from_load(new_tree, x_rl_query_ind, y_rl_ind)
        
        row = [
                    idx,
                    i,
                    rl_stats_old['pred'][0],
                    rl_stats_new['pred'][0],
                    float(rl_stats_old['real'][0]),
                    float(rl_stats_new['real'][0]), 
                    rl_stats_old['mse'],
                    rl_stats_new['mse'],
                    rl_stats_old['mae'],
                    rl_stats_new['mae'],
                    rl_stats_old['rmse'],
                    rl_stats_new['rmse'],
                    str(x_rl_tree_ind),
                    str(new_tree),
           ]

        values_exsar.append(row)
    values_final_exsar.append(row)
    c_exsar += 1
#print("values_exsar ", values_exsar)
print("len(values_exsar[0])",len(values_exsar[0]) )
print("len(values_exsar)", len(values_exsar))

In [ ]:
exsarsa_df = pd.DataFrame(values_exsar,columns=rl_columns)
exsarsa_df_no_history = exsarsa_df[no_history]
exsarsa_df_no_history.to_csv(URL + 'rl_csvs/exsarsa_df_no_history'+str(n)+'.csv',index=False)

In [ ]:
exsarsa_df_final = pd.DataFrame(values_final_exsar,columns=rl_columns)
exsarsa_df_final.to_csv(URL + 'rl_csvs/exsarsa_df_final'+str(n)+'.csv',index=False)

In [ ]:
exsarsa_df_no_history

In [ ]:
exsarsa_df_final

In [ ]:
1+1

### Archivos enteros

In [ ]:
import os
path = "/home/school/math/final_analysis.csv"

In [ ]:
os.remove(URL + 'rl_csvs/ql_df.csv')

In [ ]:
os.remove(URL + 'rl_csvs/sarsa_df.csv')

In [ ]:
os.remove(URL + 'rl_csvs/neo_df.csv')

In [ ]:
os.remove(URL + 'rl_csvs/exsarsa_df.csv')

In [ ]:
ql_df.to_csv(URL + 'rl_csvs/ql_df.csv',index=False)

In [ ]:
sarsa_df.to_csv(URL + 'rl_csvs/sarsa_df.csv',index=False)

In [ ]:
neo_df.to_csv(URL + 'rl_csvs/neo_df.csv',index=False)

In [ ]:
exsarsa_df.to_csv(URL + 'rl_csvs/exsarsa_df.csv',index=False)